___

# [ Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## GeoPandas

This notebook introduces how to work with vectors in in Python using GeoPandas https://geopandas.org library. GeoPandas extends the popular data science library pandas by adding support for geospatial data.


It covers: 

    
* 1. Reading and writing vector data into GeoDataFrame and file system 

* 2. Simple accessors functions 

* 3. Plotting maps

* 4. Geometry constructors 


The core data structure in GeoPandas is the `geopandas.GeoDataFrame`, a subclass of `pandas.DataFrame`, that can store `geometry` columns and perform spatial operations. The `geopandas.GeoSeries`, a subclass of `pandas.Series`, handles the `geometries`. 

You can have as many columns with geometries as you wish; there’s no limit typical for desktop GIS software.
 
 

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://geopandas.org/en/stable/_images/dataframe.svg")

Each `GeoSeries` can contain any `geometry type` (you can even mix them within a single array) and has a `GeoSeries.crs` attribute, which stores information about the projection (CRS stands for Coordinate Reference System). Therefore, each `GeoSeries` in a `GeoDataFrame` can be in a different projection, allowing you to have, for example, multiple versions (different projections) of the same `geometry`.

Only one `GeoSeries` in a `GeoDataFrae` is considered the `active geometry`, which means that all geometric operations applied to a `GeoDataFrame` operate on this active column.

## Reading and writing files

In [ ]:
import os 
import geopandas
import pandas as pd

`geopandas.read_file()`

In [ ]:
# reading
path_to_data = geopandas.datasets.get_path("nybb")
gdf = geopandas.read_file(path_to_data)

gdf.head()

`GeoDataFrame.to_file()`

The default file format is Shapefile, but you can specify your own with the driver keyword.

In [ ]:
# writing
path_lfs = './data'
gdf.to_file(os.path.join(path_lfs, 'my_file.geojson'), driver="GeoJSON")

## Simple accessors and methods

### Measuring area
To measure the area of each polygon (or MultiPolygon in this specific case), access the GeoDataFrame.area attribute, which returns a pandas.Series. Note that `GeoDataFrame.area` is just `GeoSeries.area` applied to the active `geometry` column.

In [ ]:
# first setting index
gdf = gdf.set_index("BoroName")

In [ ]:
# add new attribute area
gdf["area"] = gdf.area / 1000000 # km2 
gdf["area"].head()

### Getting polygon boundary and centroid

In [ ]:
gdf['boundary'] = gdf.boundary
gdf['boundary']

In [ ]:
gdf['centroid'] = gdf.centroid
gdf['centroid']

In [ ]:
# coordinates of the first centroid
gdf['centroid'].iloc[0].x, gdf['centroid'].iloc[0].y

### Measuring distance

In [ ]:
first_point = gdf['centroid'].iloc[0]
gdf['distance'] = gdf['centroid'].distance(first_point)
gdf['distance']

## Plotting maps

In [ ]:
gdf.plot("area", legend=True, figsize=(10,10))

In [ ]:
# convert ot Web Mercator
gdf_wm = gdf.to_crs(epsg=3857)
gdf_wm.crs

In [ ]:
import contextily as cx
ax = gdf_wm.plot(alpha=0.5, edgecolor='k', figsize=(10,10), legend=True)
cx.add_basemap(ax, zoom=12, source=cx.providers.Stamen.TonerLite)
# cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels)

In [ ]:
# Leaflet - folium 
# You can also explore your data interactively using GeoDataFrame.explore(), 
# which behaves in the same way plot() does but returns an interactive map instead.

# gdf.explore("area", legend=False)

## Geometry creation (constructor)

In [ ]:
gdf.head()

In [ ]:
# prepare ne df with coordinates
x = gdf['centroid'].x
y = gdf['centroid'].y
xy=pd.concat([x,y],axis=1)
xy.columns = ['x', 'y']

In [ ]:
xy.head()

In [ ]:
xy_geometry = geopandas.GeoDataFrame(
                xy, geometry=geopandas.points_from_xy(xy['x'], xy['y']))
xy_geometry.head()

#### Buffer

In [ ]:
xy_geometry["buffer"] = xy_geometry.buffer(10000)
xy_geometry.head()

In [ ]:
ax = xy_geometry["buffer"].plot(alpha=.5)
xy_geometry["geometry"].plot(ax=ax, color="red") 